In [ ]:
%matplotlib nbagg
%load_ext line_profiler
%load_ext memory_profiler

import jungfrau_utils as ju
import numpy as np
import json
import os, math
import time
from matplotlib import pyplot as plt
from matplotlib import cm
from scipy.optimize import curve_fit
from IPython.display import clear_output, display
import matplotlib.colors as colors

import h5py

from alvra_tools import *
from alvra_tools.channels import *
from alvra_tools.utils import errfunc_sigma, errfunc_fwhm, threshold
import glob

In [ ]:
# Choose a pedestal file (usually located in /sf/alvra/data/*pgroup*/res/JF_pedestals) 
# if you do not want to use the closest in time

# pedestal_file = "/sf/alvra/data/p17983/res/JF_pedestals/pedestal_20190727_2115.JF02T09V02.res.h5"
pedestal_file = None

In [ ]:
scan_name ="RuBpy_RIXS_1"     #use this for a scan
#scan_name ="RuFoil_mono_1"     #use this for a single file
pgroup ="p18442"

#######################################################################

DIR_json = "/sf/alvra/data/{}/res/scan_info/".format(pgroup)
DIR = "/sf/alvra/data/{}/raw/scan_data/{}/".format(pgroup, scan_name) 
savedir = "/das/work/p18/{}/Export_PNG/".format(pgroup)

isscan = os.path.exists(DIR)
if isscan:
    DIR = DIR
else:
    DIR = "/sf/alvra/data/{}/raw/{}/".format(pgroup, scan_name)

print (DIR)
files_scan = glob.glob(DIR_json + scan_name + '.*')

try:
    scan_range = '.'+files_scan[-1].split('.')[-2]
    print ('Scan range runs: {}'.format(scan_range))
    !ls -lah -rt $DIR | grep JF*
    !hostname
except:
    print ('Single file, not a scan')
    !ls -lah -rt $DIR | grep JF*
    !hostname


In [ ]:
# First we load the first images of a file to check if everything is fine with the pedestal &  files (check high True or False)
# We can see also how the ROIs look like (will be useful later)

clock = Clock()

fileName = "run_001507.JF02T09V02.h5"

print(DIR + fileName)

nframes = 10

total = 0
total_thr = 0

with ju.File(DIR + fileName, pedestal_file=pedestal_file) as juf:
    pulse_ids = juf["pulse_id"][:]
    print ('Total number of images is {}, load the first {}'.format(len(pulse_ids), nframes))

    for i in range(nframes):
        image = juf[i, :]
        image_thr = image.copy()
        image_thr[image < 2] = 0
        total_thr += image_thr
        total += image
        #total /= nframes
    
print (total_thr.shape)
print ("It took", clock.tick(), "seconds to process this file")

#roi = [[225, 325], [6950, 7075]]
#roi = [[225, 325], [7900, 8450]]
#roi = [[0, total_corr.shape[0]], [0, total_corr.shape[1]]]

plt.figure()
plt.subplots_adjust(wspace=0.5)
plt.subplot(121)
plt.imshow(total_thr, origin="lower", aspect='auto', vmin=0, vmax=1)
plt.title("with threshold")
plt.colorbar()
plt.subplot(122)
plt.imshow(total, origin="lower", aspect='auto', vmin=0, vmax=1)
plt.title("without threshold")
plt.colorbar()
plt.show()


In [ ]:
plt.figure()
plt.imshow(total_thr, origin="lower", aspect='auto', vmin=0, vmax=0.1)
plt.title(fileName)
plt.colorbar()
plt.show()

# New script in the next cell (single file)

In [ ]:
datalist = glob.glob(DIR + "*.JF02T09V02.h5")
datalist = sorted(datalist)
roi1 = (5000, 5200, 220, 290) # Ru ROIs
roi2 = (4600, 4800, 220, 290)
nshots = None

%mprun -f load_crop_JF_batches_on_off load_crop_JF_batches_on_off(datalist[0], roi1, roi2, 50, 25,\
                                                                  pedestal_file=pedestal_file, nshots=nshots)

In [ ]:
files = ["run_001612"]
detector = "JF02T09V02"

roi1 = (5000, 5200, 220, 290) # Ru ROIs
roi2 = (4600, 4800, 220, 290)
#roi1 = (100, 250, 140, 220)
#roi2 = (775, 925, 140, 220)

threshold_low = 2.0
threshold_high = 10

Timezero = 194.6   # position of Timezero on PP delay (in mm!)
######################################################################################
start_time = time.time()
spectrum_ON_roi1 = 0
spectrum_OFF_roi1 = 0
spectrum_ON_roi2 = 0
spectrum_OFF_roi2 = 0
Energy_eV_single = 0
Delay_mm_single = 0
        
for f in files:
    JF_single_file = DIR + f + ".{}.h5".format(detector)
    BS_single_file = DIR + f + ".BSREAD.h5"
    print("Processing file %s" % (JF_single_file))
    
#    photon_range = (6, 7)

    nshots = None  # put this to None for ALL images
    #nshots = None
       
    images_ON_roi1, images_ON_roi2, pids_ON, images_OFF_roi1, images_OFF_roi2, pids_OFF = \
    load_crop_JF_batches_on_off(JF_single_file, roi1, roi2, 50, 25, pedestal_file=pedestal_file, \
                                nshots=nshots)
    
#   images_ON_roi1, images_ON_roi2, pids_ON, images_OFF_roi1, images_OFF_roi2, pids_OFF = \
#   load_crop_JF_data_on_off(JF_single_file, roi1, roi2, 50, 25, pedestal_file=pedestal_file, \
#                            nshots=nshots)
    
    energy, pids = load_single_channel_pulseID(BS_single_file, channel_energy, 50)
    delay, pids = load_single_channel_pulseID(BS_single_file, channel_delay, 50)
    
    energy = [x for x in energy if (x > 0)]
    delay = [x for x in delay if (x > 0)]
    
    Energy_eV_single = np.mean(energy)
    Delay_mm_single = np.mean(delay)
    Delay_ps_single = (Delay_mm_single - Timezero)*2/(3e8*1e3*1e-12) # in ps, change to right t0 !
    
#    Izero_pump, Izero_unpump, energy_eV, pulseIDs = \
#    load_FEL_pp_pulseID(BS_single_file, channel_energy, 50, 25, 50)  
    
    images_ON_roi1_thr = images_ON_roi1.copy()
    images_ON_roi1_thr[images_ON_roi1 < threshold_low] = 0
    images_ON_roi1_thr[images_ON_roi1 > threshold_high] = 0
#   images_ON_roi1 = convert_to_photon_num_range(images_ON_roi1)

    images_ON_roi2_thr = images_ON_roi2.copy()
    images_ON_roi2[images_ON_roi2 < threshold_low] = 0
    images_ON_roi2[images_ON_roi2 > threshold_high] = 0
#   images_ON_roi2 = convert_to_photon_num_range(images_ON_roi2)

    images_OFF_roi1_thr = images_OFF_roi1.copy()
    images_OFF_roi1_thr[images_OFF_roi1 < threshold_low] = 0
    images_OFF_roi1_thr[images_OFF_roi1 > threshold_high] = 0
#   images_OFF_roi1 = convert_to_photon_num_range(images_OFF_roi1)

    images_OFF_roi2_thr = images_OFF_roi2.copy()
    images_OFF_roi2[images_OFF_roi2 < threshold_low] = 0
    images_OFF_roi2[images_OFF_roi2 > threshold_high] = 0
#   images_OFF_roi2 = convert_to_photon_num_range(images_OFF_roi2)

    spectrum_ON_roi1  = images_ON_roi1_thr.sum(axis = 1).sum(axis = 0)
    spectrum_OFF_roi1 = images_OFF_roi1_thr.sum(axis = 1).sum(axis = 0)
    spectrum_ON_roi2  = images_ON_roi2.sum(axis = 1).sum(axis = 0)
    spectrum_OFF_roi2 = images_OFF_roi2.sum(axis = 1).sum(axis = 0)
    
print ("Job done! It took {} seconds to process {} file".format(round((time.time() - start_time), 2),len(files)))

## Plot spectra of both ROIs together

In [ ]:
from textwrap import wrap
bin1_ = 1  # change the bin size if wanted, 1 is every pixel
bin2_ = 1  # change the bin size if wanted, 1 is every pixel

spectra_on1 = spectrum_ON_roi1.copy()
spectra_off1 = spectrum_OFF_roi1.copy()
spectra_on2 = spectrum_ON_roi2.copy()
spectra_off2 = spectrum_OFF_roi2.copy()

x_axis1 = np.arange(roi1[0], roi1[1])
x_axis2 = np.arange(roi2[0], roi2[1])

spectra_on_rebin1  = bin_sum(spectra_on1,  bin1_)
spectra_off_rebin1 = bin_sum(spectra_off1, bin1_)
x_axis_rebin1 = bin_mean(x_axis1, bin1_)
spectra_on_rebin2  = bin_sum(spectra_on2,  bin2_)
spectra_off_rebin2 = bin_sum(spectra_off2, bin2_)
x_axis_rebin2 = bin_mean(x_axis2, bin2_)

spectra_on_err1  = np.sqrt(spectra_on_rebin1)
spectra_off_err1 = np.sqrt(spectra_off_rebin1)
spectra_on_err2  = np.sqrt(spectra_on_rebin2)
spectra_off_err2 = np.sqrt(spectra_off_rebin2)

low_err1= (spectra_on_rebin1 - spectra_off_rebin1)-np.sqrt(spectra_on_err1**2+spectra_off_err1**2)
high_err1= (spectra_on_rebin1 - spectra_off_rebin1)+np.sqrt(spectra_on_err1**2+spectra_off_err1**2)
low_err2= (spectra_on_rebin2 - spectra_off_rebin2)-np.sqrt(spectra_on_err2**2+spectra_off_err2**2)
high_err2= (spectra_on_rebin2 - spectra_off_rebin2)+np.sqrt(spectra_on_err2**2+spectra_off_err2**2)

plt.figure(figsize=(10,5))
plt.suptitle(scan_name)

plt.subplot(221)

plt.gca().set_title("\n".join(wrap('ROI1: {} Energy = {} eV'.format(roi1, np.round(Energy_eV_single, 2)),30)))
#plt.gca().set_title('ROI1: [], Delay = {} ps'.format(roi1, np.round(Delay_ps_single, 2)))
plt.plot(x_axis_rebin1, spectra_on_rebin1, label="ON", marker='.')
plt.plot(x_axis_rebin1, spectra_off_rebin1, label="OFF", marker='.')
plt.legend(loc="best")
plt.grid()

plt.subplot(223)
plt.plot(x_axis_rebin1,spectra_on_rebin1 - spectra_off_rebin1, label="ON - OFF",marker='.')
plt.fill_between(x_axis_rebin1, low_err1, high_err1, color='lightblue')
plt.legend(loc="best")
plt.xlabel("JF pixel")
plt.grid()

plt.subplot(222)
plt.gca().set_title("\n".join(wrap('ROI2: {} Energy = {} eV'.format(roi2, np.round(Energy_eV_single, 2)),30)))
#plt.gca().set_title('ROI2: [], Delay = {} ps'.format(roi2, np.round(Delay_ps_single, 2)))
plt.plot(x_axis_rebin2, spectra_on_rebin2, label="ON", marker='.')
plt.plot(x_axis_rebin2, spectra_off_rebin2, label="OFF", marker='.')
plt.legend(loc="best")
plt.grid()

plt.subplot(224)
plt.plot(x_axis_rebin2,spectra_on_rebin2 - spectra_off_rebin2, label="ON - OFF",marker='.')
plt.fill_between(x_axis_rebin2, low_err2, high_err2, color='lightblue')
plt.legend(loc="best")
plt.xlabel("JF pixel")
plt.grid()

plt.show()

## Or separately: ROI1

In [ ]:
bin_ = 1   # change the bin size if wanted, 1 is every pixel

spectra_on = spectrum_ON_roi1.copy()
spectra_off = spectrum_OFF_roi1.copy()

x_axis = np.arange(roi1[0], roi1[1])

spectra_on_rebin1  = bin_sum(spectra_on,  bin_)
spectra_off_rebin1 = bin_sum(spectra_off, bin_)
x_axis_rebin1 = bin_mean(x_axis, bin_)

spectra_on_err1  = np.sqrt(spectra_on_rebin1)
spectra_off_err1 = np.sqrt(spectra_off_rebin1)



#spectrumAllOutFoc=spectra_on_rebin1 + spectra_off_rebin1
plt.figure()
plt.suptitle(scan_name + ' // ROI1: {}'.format(roi1))
plt.subplot(211)
plt.gca().set_title('Energy = {} eV'.format(np.round(Energy_eV_single, 2)))
#plt.gca().set_title('Delay = {} ps'.format(np.round(Delay_ps_single, 2)))
plt.plot(x_axis_rebin1, spectra_on_rebin1, label="ON", marker='.')
plt.plot(x_axis_rebin1, spectra_off_rebin1, label="OFF", marker='.')
plt.legend(loc="best")
plt.grid()

#plt.title(jf_file)

low_err= (spectra_on_rebin1 - spectra_off_rebin1)-np.sqrt(spectra_on_err1**2+spectra_off_err1**2)
high_err= (spectra_on_rebin1 - spectra_off_rebin1)+np.sqrt(spectra_on_err1**2+spectra_off_err1**2)
#x_axis_rebin1,
plt.subplot(212)
plt.plot(x_axis_rebin1,spectra_on_rebin1 - spectra_off_rebin1, label="ON - OFF",marker='.')
#plt.plot(spectra_on_rebin1 - spectra_off_rebin1, label="ON - OFF",marker='.')
plt.xlabel("JF pixel")
plt.grid()


plt.legend(loc="best")
plt.fill_between(x_axis_rebin1, low_err, high_err, color='lightblue')
plt.show()

## Or separately: ROI2

In [ ]:
bin_ = 1  # change the bin size if wanted, 1 is every pixel

spectra_on = spectrum_ON_roi2.copy()
spectra_off = spectrum_OFF_roi2.copy()

x_axis = np.arange(roi2[0], roi2[1])

spectra_on_rebin2  = bin_sum(spectra_on,  bin_)
spectra_off_rebin2 = bin_sum(spectra_off, bin_)
x_axis_rebin2 = bin_mean(x_axis, bin_)

spectra_on_err2  = np.sqrt(spectra_on_rebin2)
spectra_off_err2 = np.sqrt(spectra_off_rebin2)

#spectrumAllOutFoc=spectra_on_rebin1 + spectra_off_rebin1
plt.figure()
plt.suptitle(scan_name + ' // ROI1: {}'.format(roi1))
plt.subplot(211)
plt.gca().set_title('Energy = {} eV'.format(np.round(Energy_eV_single, 2)))
#plt.gca().set_title('Delay = {} ps'.format(np.round(Delay_ps_single, 2)))
plt.plot(x_axis_rebin2, spectra_on_rebin2, label="ON", marker='.')
plt.plot(x_axis_rebin2, spectra_off_rebin2, label="OFF", marker='.')
plt.legend(loc="best")
plt.grid()

#plt.title(jf_file)

low_err= (spectra_on_rebin2 - spectra_off_rebin2)-np.sqrt(spectra_on_err2**2+spectra_off_err2**2)
high_err= (spectra_on_rebin2 - spectra_off_rebin2)+np.sqrt(spectra_on_err2**2+spectra_off_err2**2)
#x_axis_rebin1,
plt.subplot(212)
plt.plot(x_axis_rebin2,spectra_on_rebin2 - spectra_off_rebin2, label="ON - OFF",marker='.')
plt.xlabel("JF pixel")
plt.grid()

#plt.plot(spectra_on_rebin1 - spectra_off_rebin1, label="ON - OFF",marker='.')

plt.legend(loc="best")
plt.fill_between(x_axis_rebin2, low_err, high_err, color='lightblue')
plt.show()

# New script in the next cell (scans) 

In [ ]:
print (DIR_json + scan_name + scan_range + ".json")

In [ ]:
scan_name = "RuBpy_RIXS_1"          # you need to change this name, but run also cell #3 above to get scan_range
detector = "JF02T09V02"

roi1 = (5000, 5200, 220, 290)
roi2 = (4600, 4800, 220, 290) #Ru ROIs
#roi1 = (6450, 6550, 220, 300)
#roi2 = (5450, 5700, 220, 300)
threshold_low = 2.0
threshold_high = 10

######################################################################################

fname_scan = DIR_json + scan_name + scan_range + ".json"
scan_data = load_scan_data(fname_scan)


Energy_eV = []
Delay_mm = []
XAS_roi1 = []
XAS_roi1_on = []
#Error_XAS_roi1 = []
XAS_roi2 = []
XAS_roi2_on = []
#Error_XAS_roi2 = []
RIXS_transient_roi1 = []
RIXS_transient_roi2 = []
Error_transient_roi1 = []
Error_transient_roi2 = []
RIXS_unpumped_roi1 = []
RIXS_unpumped_roi2 = []
Error_unpumped_roi1 = []
Error_unpumped_roi2 = []
RIXS_pumped_roi1 = []
RIXS_pumped_roi2 = []
Error_pumped_roi1 = []
Error_pumped_roi2 = []
XAS_transient_roi1=[]
XES_transient_roi1 =[]
XAS_transient_roi2=[]
XES_transient_roi2 =[]

#XES_1D = []

clock = Clock()
num_scan_files = len(scan_data)
print("\nProcessing", num_scan_files, "total files")

for i, (bs, fn) in enumerate(scan_data):
#for i in range(0,4):
    print("\n File {} out of {}: Processing {}".format(i+1, num_scan_files, fn.split('/')[-2:]))
    
    spectrum_ON_roi1 = 0
    spectrum_OFF_roi1 = 0
    spectrum_ON_roi2 = 0
    spectrum_OFF_roi2 = 0
    
    nshots = None  # put this to None for ALL images
    #nshots = None
    
    
    images_ON_roi1, images_ON_roi2, pids_ON, images_OFF_roi1, images_OFF_roi2, pids_OFF = \
    load_crop_JF_batches_on_off(fn, roi1, roi2, 50, 25, pedestal_file=pedestal_file, nshots=nshots)
    
#   images_ON_roi1, images_ON_roi2, pids_ON, images_OFF_roi1, images_OFF_roi2, pids_OFF = \
#   load_crop_JF_data_on_off(fn, roi1, roi2, 50, 25, pedestal_file=pedestal_file, nshots=nshots)
    
    energy, pids = load_single_channel_pulseID(bs, channel_energy, 50)
    energy = [x for x in energy if (x > 0)]
    Energy_eV.append(np.mean(energy))
    
    delay, pids = load_single_channel_pulseID(bs, channel_delay, 50)
    delay = [x for x in delay if (x > 0)]
    Delay_mm.append(np.mean(delay))
    
    images_ON_roi1_thr = images_ON_roi1.copy()
    images_ON_roi1_thr[images_ON_roi1 < threshold_low] = 0
    images_ON_roi1_thr[images_ON_roi1 > threshold_high] = 0
    
    images_OFF_roi1_thr = images_OFF_roi1.copy()
    images_OFF_roi1_thr[images_OFF_roi1 < threshold_low] = 0
    images_OFF_roi1_thr[images_OFF_roi1 > threshold_high] = 0
    
    images_ON_roi2_thr = images_ON_roi2.copy()
    images_ON_roi2_thr[images_ON_roi2 < threshold_low] = 0
    images_ON_roi2_thr[images_ON_roi2 > threshold_high] = 0
    
    images_OFF_roi2_thr = images_OFF_roi2.copy()
    images_OFF_roi2_thr[images_OFF_roi2 < threshold_low] = 0
    images_OFF_roi2_thr[images_OFF_roi2 > threshold_high] = 0
    
    spectrum_ON_roi1 = images_ON_roi1_thr.sum(axis = 1).sum(axis = 0)
    spectrum_OFF_roi1 = images_OFF_roi1_thr.sum(axis = 1).sum(axis = 0)
    spectrum_ON_roi2 = images_ON_roi2_thr.sum(axis = 1).sum(axis = 0)
    spectrum_OFF_roi2 = images_OFF_roi2_thr.sum(axis = 1).sum(axis = 0)

######################### ROI 1 ##############################################
    
    XAS_roi1.append(np.abs(spectrum_OFF_roi1.sum()))
    XAS_roi1_on.append(np.abs(spectrum_ON_roi1.sum()))
    RIXS_unpumped_roi1.append(spectrum_OFF_roi1)
    Error_unpumped_roi1.append(np.sqrt(spectrum_OFF_roi1))
    RIXS_pumped_roi1.append(spectrum_ON_roi1)
    Error_pumped_roi1.append(np.sqrt(spectrum_ON_roi1))
    RIXS_transient_roi1.append(spectrum_ON_roi1 - spectrum_OFF_roi1)
    Error_transient_roi1.append(np.sqrt((np.sqrt(spectrum_ON_roi1))**2 + (np.sqrt(spectrum_OFF_roi1))**2))
    
    XES_transient_roi1.append(np.abs(spectrum_ON_roi1 - spectrum_OFF_roi1))
 #   XES_1D.append(np.sum(XES_transient_roi1, axis = 1))
    
    XAS_transient_roi1.append(np.abs(np.sum(RIXS_transient_roi1)))
    
######################### ROI 2 ##############################################
    
    XAS_roi2.append(np.abs(spectrum_OFF_roi2.sum()))
    XAS_roi2_on.append(np.abs(spectrum_ON_roi2.sum()))
    RIXS_unpumped_roi2.append(spectrum_OFF_roi2)
    Error_unpumped_roi2.append(np.sqrt(spectrum_OFF_roi2))
    RIXS_pumped_roi2.append(spectrum_ON_roi2)
    Error_pumped_roi2.append(np.sqrt(spectrum_ON_roi2))
    RIXS_transient_roi2.append(spectrum_ON_roi2 - spectrum_OFF_roi2)
    Error_transient_roi2.append(np.sqrt((np.sqrt(spectrum_ON_roi2))**2 + (np.sqrt(spectrum_OFF_roi2))**2))
    
    XES_transient_roi2.append(np.abs(spectrum_ON_roi2 - spectrum_OFF_roi2))
#    XES_1D.append(np.sum(XES_transient_roi2, axis = 1))
    
    XAS_transient_roi2.append(np.abs(np.sum(RIXS_transient_roi2)))
    clear_output(wait=True)
    
    print ("It took {} to process file {}".format(clock.tick(), fn.split('/')[-1]))

Energy_eV = np.array(Energy_eV)
Delay_mm = np.array(Delay_mm)
XAS_roi1 = np.array(XAS_roi1)
XAS_roi1_on = np.array(XAS_roi1_on)
RIXS_unpumped_roi1 = np.array(RIXS_unpumped_roi1)
Error_unpumped_roi1 = np.array(Error_unpumped_roi1)
RIXS_pumped_roi1 = np.array(RIXS_pumped_roi1)
Error_pumped_roi1 = np.array(Error_pumped_roi1)
RIXS_transient_roi1 = np.array(RIXS_transient_roi1)
Error_transient_roi1 = np.array(Error_transient_roi1)
XAS_transient_roi1=np.array(XAS_transient_roi1)
XES_transient_roi1=np.array(XES_transient_roi1)
#XES_1D=np.array(XES_1D)
XAS_roi2 = np.array(XAS_roi2)
XAS_roi2_on = np.array(XAS_roi2_on)
RIXS_unpumped_roi2 = np.array(RIXS_unpumped_roi2)
Error_unpumped_roi2 = np.array(Error_unpumped_roi2)
RIXS_pumped_roi2 = np.array(RIXS_pumped_roi2)
Error_pumped_roi2 = np.array(Error_pumped_roi2)
RIXS_transient_roi2 = np.array(RIXS_transient_roi2)
Error_transient_roi2 = np.array(Error_transient_roi2)
XAS_transient_roi2=np.array(XAS_transient_roi2)
XES_transient_roi2=np.array(XES_transient_roi2)
#XES_1D=np.array(XES_1D)

print ("\nJob done! It took", clock.tock(), "seconds to process", num_scan_files, "file(s)")



In [ ]:
outdir = '/das/work/p18/p18442/Exported_data/'
filename_h5 = outdir + scan_name + "_export.h5" 
with h5py.File(filename_h5, "w") as file2write:
    file2write.create_dataset("energy_eV", data = Energy_eV)
    file2write.create_dataset("roi1", data = roi1)
    file2write.create_dataset("pump_roi1", data = RIXS_pumped_roi1)
    file2write.create_dataset("unpump_roi1", data = RIXS_unpumped_roi1)
    file2write.create_dataset("transient_roi1", data = RIXS_transient_roi1)
    file2write.create_dataset("roi2", data = roi2)
    file2write.create_dataset("pump_roi2", data = RIXS_pumped_roi2)
    file2write.create_dataset("unpump_roi2", data = RIXS_unpumped_roi2)
    file2write.create_dataset("transient_roi2", data = RIXS_transient_roi2)

print(filename_h5)

In [ ]:
scan_name_load = 'RuBpy_RIXS_6'

##########################################################

outdir = '/das/work/p18/p18442/Exported_data/'
fn = outdir + scan_name_load + "_export.h5" 
print (fn)
with h5py.File(fn, "r") as file2open:
    Energy_eV = file2open['energy_eV'][:]
    roi1 = file2open['roi1'][:]
    RIXS_pumped_roi1 = file2open['pump_roi1'][:]
    RIXS_unpumped_roi1 = file2open['unpump_roi1'][:]
    RIXS_transient_roi1 = file2open['transient_roi1'][:]
    roi2 = file2open['roi2'][:]
    RIXS_pumped_roi2 = file2open['pump_roi2'][:]
    RIXS_unpumped_roi2 = file2open['unpump_roi2'][:]
    RIXS_transient_roi2 = file2open['transient_roi2'][:]

### For no pump probe data:

In [ ]:
RIXS_total = RIXS_unpumped_roi1 + RIXS_pumped_roi1

plt.figure(figsize=(8,4))
plt.imshow(RIXS_total, aspect = 'auto', cmap = "bwr", origin ="lower", extent=[roi1[0], roi1[1], Energy_eV[0], Energy_eV[-1]])
plt.xlabel("Emission energy (pixel)")
plt.ylabel("Incident energy (eV)")

plt.title(scan_name_load)
plt.colorbar()
plt.show()


### For pump probe data (ROI1 and ROI2 together):

In [ ]:
plt.figure(figsize=(10,8))
plt.suptitle(scan_name_load)
plt.subplots_adjust(hspace=0.5)

plt.subplot(321)
plt.imshow(RIXS_unpumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi1[0], roi1[1], Energy_eV[0], Energy_eV[-1]])
plt.title("RIXS unpumped roi1")
plt.colorbar()

plt.subplot(322)
plt.imshow(RIXS_unpumped_roi2, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi2[0], roi2[1], Energy_eV[0], Energy_eV[-1]])#, vmin=-250, vmax=250)
plt.title("RIXS unpumped roi2")
plt.colorbar()

plt.subplot(323)
plt.imshow(RIXS_pumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi1[0], roi1[1], Energy_eV[0], Energy_eV[-1]])#, vmin=-0.03, vmax=0.03)
plt.title("RIXS pumped roi1")
plt.colorbar()

plt.subplot(324)
plt.imshow(RIXS_pumped_roi2, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi2[0], roi2[1], Energy_eV[0], Energy_eV[-1]])
plt.title("RIXS pumped roi2")
plt.colorbar()

plt.subplot(325)
plt.imshow(RIXS_transient_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi1[0], roi1[1], Energy_eV[0], Energy_eV[-1]], vmin=-250, vmax=250)
plt.title("RIXS transient roi1")
plt.colorbar()

plt.subplot(326)
plt.imshow(RIXS_transient_roi2, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi2[0], roi2[1], Energy_eV[0], Energy_eV[-1]], vmin=-250, vmax=250)
plt.title("RIXS transient roi2")
plt.colorbar()



plt.show()

### Only ROI1

In [ ]:
plt.figure(figsize=(5,9))
plt.suptitle(scan_name_load)
plt.subplots_adjust(hspace=0.5)

plt.subplot(311)
plt.imshow(RIXS_unpumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi1[0], roi1[1], Energy_eV[0], Energy_eV[-1]])
plt.title("RIXS unpumped roi1")
plt.colorbar()
plt.show()

plt.subplot(312)
plt.imshow(RIXS_pumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi1[0], roi1[1], Energy_eV[0], Energy_eV[-1]])#, vmin=-250, vmax=250)
plt.title("RIXS pumped roi1")
plt.colorbar()
plt.show()

plt.subplot(313)
plt.imshow(RIXS_transient_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi1[0], roi1[1], Energy_eV[0], Energy_eV[-1]], vmin=-250, vmax=250)
plt.title("RIXS transient roi1")
plt.colorbar()
plt.show()

### Only ROI2

In [ ]:
plt.figure(figsize=(5,9))
plt.suptitle(scan_name_load)
plt.subplots_adjust(hspace=0.5)

plt.subplot(311)
plt.imshow(RIXS_unpumped_roi2, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi2[0], roi2[1], Energy_eV[0], Energy_eV[-1]])
plt.title("RIXS unpumped roi2")
plt.colorbar()
plt.show()

plt.subplot(312)
plt.imshow(RIXS_pumped_roi2, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi2[0], roi2[1], Energy_eV[0], Energy_eV[-1]])#, vmin=-250, vmax=250)
plt.title("RIXS pumped roi2")
plt.colorbar()
plt.show()

plt.subplot(313)
plt.imshow(RIXS_transient_roi2, aspect = 'auto', cmap = "bwr", origin ="lower", \
           extent=[roi2[0], roi2[1], Energy_eV[0], Energy_eV[-1]], vmin=-250, vmax=250)
plt.title("RIXS transient roi2")
plt.colorbar()
plt.show()

## XAS plots in PFY

### ROI1

In [ ]:
XAS_pump_roi1 =  np.sum(RIXS_pumped_roi1, axis=1)
XAS_unpump_roi1 = np.sum(RIXS_unpumped_roi1, axis=1)
XAS_pumpprobe_roi1 = XAS_pump_roi1 - XAS_unpump_roi1

plt.figure(figsize=(10,5))
plt.suptitle(scan_name_load + '    XAS, roi1')
plt.subplot(121)
plt.plot(Energy_eV,XAS_pump_roi1, label="laser on", marker='.')
plt.plot(Energy_eV,XAS_unpump_roi1, label="laser off", marker='.')
plt.legend(loc='best')
plt.xlabel("Incident energy (eV)")
plt.grid()
plt.ylabel("Counts")

plt.subplot(122)
plt.plot(Energy_eV,XAS_pumpprobe_roi1, label="pump probe",color='green', marker='.')
plt.legend(loc='best')
plt.xlabel("Incident energy (eV)")
plt.grid()
plt.show()

### ROI2

In [ ]:
XAS_pump_roi2 =  np.sum(RIXS_pumped_roi2, axis=1)
XAS_unpump_roi2 = np.sum(RIXS_unpumped_roi2, axis=1)
XAS_pumpprobe_roi2 = XAS_pump_roi2 - XAS_unpump_roi2

plt.figure(figsize=(10,5))
plt.suptitle(scan_name_load + '    XAS, roi2')
plt.subplot(121)
plt.plot(Energy_eV,XAS_pump_roi2, label="laser on", marker='.')
plt.plot(Energy_eV,XAS_unpump_roi2, label="laser off", marker='.')
plt.legend(loc='best')
plt.xlabel("Incident energy (eV)")
plt.grid()
plt.ylabel("Counts")

plt.subplot(122)
plt.plot(Energy_eV,XAS_pumpprobe_roi2, label="pump probe",color='green', marker='.')
plt.legend(loc='best')
plt.xlabel("Incident energy (eV)")
plt.grid()
plt.show()

In [ ]:
Timezero = 194.6   # position of Timezero on PP delay (in mm!)
Delay_ps = (Delay_mm - Timezero)*2/(3e8*1e3*1e-12) # change to right t0 !

plt.figure(figsize=(8,4))
plt.plot(Delay_ps, XES_transient_roi1.sum(axis = 1), marker ='.')
plt.xlabel("delay (ps)")
plt.ylabel("Incident energy (eV)")

plt.title(scan_name)
plt.show()


In [ ]:
Timezero = 194.6   # position of Timezero on PP delay (in mm!)
Delay_ps = (Delay_mm - Timezero)*2/(3e8*1e3*1e-12) # change to right t0 !

plt.figure()
#plt.plot(Energy_eV,RIXS_transient_roi1, label="", marker='.')
plt.plot(Delay_ps,XES_1D, label="", marker='.')
plt.xlabel("Delay (ps)")
plt.ylabel("Counts")
plt.show

In [ ]:
plt.figure(figsize=(5,9))
plt.subplots_adjust(hspace=0.5)

plt.subplot(311)
plt.imshow(RIXS_unpumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", extent=[roi1[0], roi1[1], Delay_ps[0], Delay_ps[-1]])
plt.title("RIXS unpumped")
plt.colorbar()
plt.show()

plt.subplot(312)
plt.imshow(RIXS_pumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", extent=[roi1[0], roi1[1], Delay_ps[0], Delay_ps[-1]])#, vmin=-250, vmax=250)
plt.title("RIXS pumped")
plt.colorbar()
plt.show()

plt.subplot(313)
plt.imshow(RIXS_transient_roi1, aspect = 'auto', cmap = "rainbow", origin ="lower", extent=[roi1[0], roi1[1], Delay_ps[0], Delay_ps[-1]])#, vmin=-0.03, vmax=0.03)
plt.title("RIXS transient")
plt.colorbar()
plt.show()

In [ ]:
np.sum(np.abs(RIXS_transient_roi1))

In [ ]:
plt.figure()
#plt.plot(Energy_eV,RIXS_transient_roi1, label="", marker='.')
plt.plot(Energy_eV,XAS_roi1_on-XAS_roi1, label="", marker='.')
plt.xlabel("Incident energy (eV)")
plt.ylabel("Counts")
plt.show

plt.figure()
plt.plot(x_axis,RIXS_transient_roi1.sum(axis=0), label="", marker='.')
plt.xlabel("Pixels")
plt.ylabel("Counts")
plt.show

plt.figure()
plt.plot(Delay_ps,np.abs(RIXS_transient_roi1).sum(axis=1), label="", marker='.')
plt.xlabel("Delay ps")
plt.ylabel("Counts")
plt.show

In [ ]:
Timezero = 194.6   # position of Timezero on PP delay (in mm!)
Delay_ps = (Delay_mm - Timezero)*2/(3e8*1e3*1e-12) # change to right t0 !

plt.figure()
#plt.plot(Energy_eV,RIXS_transient_roi1, label="", marker='.')
plt.plot(Delay_ps,XAS_roi1_on-XAS_roi1, label="", marker='.')
plt.xlabel("Delay (ps)")
plt.ylabel("Counts")
plt.show

In [ ]:
plt.figure(figsize=(5,9))
plt.subplots_adjust(hspace=0.5)

plt.subplot(311)
plt.imshow(RIXS_unpumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", extent=[roi1[0], roi1[1], Delay_mm[0], Energy_eV[-1]])
plt.title("RIXS unpumped")
plt.colorbar()
plt.show()

plt.subplot(312)
plt.imshow(RIXS_pumped_roi1, aspect = 'auto', cmap = "bwr", origin ="lower", extent=[roi1[0], roi1[1], Delay_mm[0], Energy_eV[-1]])#, vmin=-250, vmax=250)
plt.title("RIXS pumped")
plt.colorbar()
plt.show()

plt.subplot(313)
plt.imshow(RIXS_transient_roi1, aspect = 'auto', cmap = "rainbow", origin ="lower", extent=[roi1[0], roi1[1], Delay_mm[0], Energy_eV[-1]])#, vmin=-0.03, vmax=0.03)
plt.title("RIXS transient")
plt.colorbar()
plt.show()

In [ ]:
savedir = '/das/work/p18/p18387/exported_data/'
filename_txt = savedir + "run001710_Ka.txt" 
output_matrix =[]
output_matrix = np.array(x_axis)
output_matrix = np.vstack((output_matrix, np.array(spectrum_ON_roi1)))
output_matrix = np.vstack((output_matrix, np.array(spectrum_OFF_roi1)))
#output_matrix = np.array()
#output_matrix = np.vstack((output_matrix, np.array(spectrum_ON_roi2)))
#output_matrix = np.vstack((output_matrix, np.array(spectrum_OFF_roi2)))

np.savetxt(filename_txt, np.transpose(output_matrix))

In [ ]:
def edge_removal(module_edge, roi_removal, array):
    index_edge = module_edge - roi_removal[0]
    array_input = array.copy()
    print (array[index_edge-1:index_edge+3])
    array[index_edge] = array[index_edge-1]/2
    array[index_edge-1] = array[index_edge-1]/2

    array[index_edge+1] = array[index_edge+2]/2
    array[index_edge+2] = array[index_edge+2]/2 
    print (array[index_edge-1:index_edge+3])
    return array, array_input

In [ ]:
spectrum_ON_roi1, spectrum_ON_roi1_orig = edge_removal(6472, roi1, spectrum_ON_roi1) 
spectrum_OFF_roi1, spectrum_OFF_roi1_orig = edge_removal(6472, roi1, spectrum_OFF_roi1) 